In [ ]:
%pip install duckduckgo-search

In [ ]:
from duckduckgo_search import DDGS #DuckDuckGo has changed the api so we need to update
from fastcore.all import *

def search_images(keywords, max_images=200): return L(DDGS().images(keywords, max_results=max_images)).itemgot('image')
import time, json

In [ ]:
urls = search_images('durian', max_images=1)
urls[0]

In [ ]:
from fastdownload import download_url
dest = 'durian.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
download_url(search_images('durian', max_images=1)[0], 'durian.jpg', show_progress=False)
Image.open('durian.jpg').to_thumb(256,256)

In [ ]:
# Download durian images to external data folder
from fastai.vision.all import *

# Path to external data folder (one directory up from project)
data_path = Path('../data')
durian_dest = data_path/'durian'

# Create durian directory if it doesn't exist
durian_dest.mkdir(exist_ok=True, parents=True)

# Download durian images
print(f"Downloading durian images to {durian_dest}")
durian_urls = search_images('durian', max_images=300)
download_images(durian_dest, urls=durian_urls)
time.sleep(2)

# Resize images
print("Resizing durian images...")
resize_images(durian_dest, max_size=400, dest=durian_dest)

print(f"Downloaded {len(get_image_files(durian_dest))} durian images")

In [ ]:
# Verify images in external data folder
data_path = Path('../data')
failed = verify_images(get_image_files(data_path))
failed.map(Path.unlink)
print(f"Removed {len(failed)} corrupted images")
print(f"Categories found: {[d.name for d in data_path.iterdir() if d.is_dir()]}")

In [ ]:
# Create DataLoaders for 3-class classification
data_path = Path('../data')

dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(data_path, bs=32)

print(f"Classes: {dls.vocab}")
print(f"Training samples: {len(dls.train_ds)}")
print(f"Validation samples: {len(dls.valid_ds)}")

dls.show_batch(max_n=6)

In [ ]:
# Train the model
learn = vision_learner(dls, resnet34, metrics=error_rate)
print("Training model...")
learn.fine_tune(4)

In [ ]:
# Test model prediction
# Replace 'durian.jpg' with any image file you want to test
test_image = 'images.jpeg'
pred_class, pred_idx, probs = learn.predict(PILImage.create(test_image))

print(f"Predicted class: {pred_class}")
print(f"Confidence scores:")
for i, label in enumerate(dls.vocab):
    print(f"  {label}: {probs[i]:.4f}")
    
# Jangle's verdict
if pred_class == 'durian':
    verdict = "Jangle will NOT eat this! (It's durian)"
elif pred_class == 'inedible':
    verdict = "Jangle will NOT eat this! (It's not food)"
else:  # non_durian_edible
    verdict = "Jangle will eat this!"
    
print(f"\nVerdict: {verdict}")

In [ ]:
# Export the trained model
learn.export('model.pkl')
print("Model exported as model.pkl")

# Copy to api folder for deployment
import shutil
shutil.copy('model.pkl', 'api/model.pkl')
print("Model copied to api/model.pkl for deployment")